# Modelos listos para submission

In [1]:
from sklearn.model_selection import train_test_split, cross_val_score
import pandas as pd

In [2]:
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np

MODEL = "distilbert-videogame-descriptions-rating"

tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

def sentence_clf_output(text):
    """retorna el SequenceClassifierOutput"""
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input, return_dict=True, output_hidden_states=True)
    return output

def logits_embedding(clf_output):
    # retorna el vector de scores de clasificacion (antes de la capa softmax)
    return clf_output['logits'][0].detach().numpy().reshape(1,5)

In [3]:
def integrar_bert_logits(df_in):
    df = df_in.copy(deep=True)

    embed = lambda row: logits_embedding(sentence_clf_output(row))
    bert_logits = np.concatenate(df['short_description'].apply(embed).to_numpy())  # .reshape(100,3)

    df[['bert1','bert2','bert3','bert4','bert5']] = pd.DataFrame(bert_logits, index= df.index)

    return df

In [4]:
df_train = pd.read_pickle('train.pickle')
df_train = integrar_bert_logits(df_train)

In [5]:
from sklearn.base import BaseEstimator, TransformerMixin
from preprocessing import Nothing, CategoriesTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, PowerTransformer, OneHotEncoder
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.pipeline import Pipeline
import re

class Nothing(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self,X):
        return X


class CategoriesTokenizer:
    def __init__(self):
        pass

    def __call__(self, doc):
        return doc.split(';')

boc_some_values = CountVectorizer(
    tokenizer = CategoriesTokenizer(),
    max_df = 1.0,
    min_df = 0.05
    )


boc_many_values = CountVectorizer(
    tokenizer = CategoriesTokenizer(),
    max_df = 1.0,
    min_df = 1
    )


def custom_features(dataframe_in):
    df = dataframe_in.copy(deep=True)

    df['month'] = pd.to_datetime(df['release_date']).dt.month
    df['release_date'] = pd.to_datetime(df['release_date']).apply(lambda x: x.to_julian_date())

    df['revenue'] = pd.Series([0 for _ in range(len(dataframe_in))])

    df.loc[df.publisher.str.match('.*microsoft.*', flags=re.IGNORECASE).values, 'revenue'] = 10.260
    df.loc[df.publisher.str.match('.*netease.*', flags=re.IGNORECASE).values, 'revenue'] = 6.668
    df.loc[df.publisher.str.match('.*activision.*', flags=re.IGNORECASE).values, 'revenue'] = 6.388
    df.loc[df.publisher.str.match('.*electronic.*', flags=re.IGNORECASE).values, 'revenue'] = 5.537
    df.loc[df.publisher.str.match('.*bandai.*', flags=re.IGNORECASE).values, 'revenue'] = 3.018
    df.loc[df.publisher.str.match('.*square.*', flags=re.IGNORECASE).values, 'revenue'] = 2.386
    df.loc[df.publisher.str.match('.*nexon.*', flags=re.IGNORECASE).values, 'revenue'] = 2.286
    df.loc[df.publisher.str.match('.*ubisoft.*', flags=re.IGNORECASE).values, 'revenue'] = 1.446
    df.loc[df.publisher.str.match('.*konami.*', flags=re.IGNORECASE).values, 'revenue'] = 1.303
    df.loc[df.publisher.str.match('.*SEGA.*').values, 'revenue'] = 1.153
    df.loc[df.publisher.str.match('.*capcom.*', flags=re.IGNORECASE).values, 'revenue'] = 0.7673
    df.loc[df.publisher.str.match('.*warner.*', flags=re.IGNORECASE).values, 'revenue'] = 0.7324

    return df


preprocessing_bert = ColumnTransformer(
    transformers=[
        ('BoC-plat',boc_some_values,'platforms'),
        ('BoC-cat',boc_some_values,'categories'),
        ('BoC-genres',boc_some_values,'genres'),
        ('BoC-tags',boc_some_values,'tags'),

        ('BoC-dev',boc_many_values,'developer'),
        ('BoC-pub',boc_many_values,'publisher'),

        ('OneHotEncoder',OneHotEncoder(handle_unknown='ignore'),['month']),
        ('MinMaxScaler',MinMaxScaler(),['required_age','price','release_date']),
        ('BoxCox',PowerTransformer(method='yeo-johnson'),['achievements','average_playtime','revenue']),
        ('unchanged',Nothing(),['english','bert1','bert2','bert3','bert4','bert5'])
])

In [6]:
df_train = custom_features(df_train)

In [7]:
def make_pipeline(clf,prepro):
    pipeline = Pipeline(
        [("procesamiento", prepro),
        ("selector", SelectPercentile(f_classif, percentile=95)),
        ("classifier", clf)]
    )
    return pipeline

In [8]:
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.neural_network import MLPClassifier

In [9]:
clf = VotingClassifier(estimators=[
        ('SVC', LinearSVC(random_state=0)),
        ('RF', RandomForestClassifier(n_estimators=250, random_state=0)),
        ('MLP', MLPClassifier(hidden_layer_sizes=(200,), learning_rate_init=0.01, solver='sgd',random_state = 0))
    ])

Entrenando con todos los datos

In [10]:
import numpy as np
np.seterr(divide='ignore', invalid='ignore');

In [11]:
%%capture
pipe_clf = make_pipeline(clf,preprocessing_bert)
score = cross_val_score(pipe_clf,df_train, df_train['rating'],scoring='f1_weighted')

In [12]:
score

array([0.37381541, 0.36584307, 0.37928127, 0.37928575, 0.35647289])

In [13]:
np.mean(score)

0.3709396771749649

In [14]:
pipe_clf = make_pipeline(clf,preprocessing_bert)
pipe_clf.fit(df_train, df_train['rating'])

/home/camilo/miniconda3/envs/prog_cientifica/lib/python3.10/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/camilo/miniconda3/envs/prog_cientifica/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Pipeline(steps=[('procesamiento',
                 ColumnTransformer(transformers=[('BoC-plat',
                                                  CountVectorizer(min_df=0.05,
                                                                  tokenizer=<__main__.CategoriesTokenizer object at 0x7f40ac399c90>),
                                                  'platforms'),
                                                 ('BoC-cat',
                                                  CountVectorizer(min_df=0.05,
                                                                  tokenizer=<__main__.CategoriesTokenizer object at 0x7f40ac399c90>),
                                                  'categories'),
                                                 ('BoC-genres',
                                                  CountVectorizer(min_df=0.05,
                                                                  tokenizer...
                                                 ('unchanged', Nothing(),
                                                  ['english', 'bert1', 'bert2',
                                                   'bert3', 'bert4',
                                                   'bert5'])])),
                ('selector', SelectPercentile(percentile=95)),
                ('classifier',
                 VotingClassifier(estimators=[('SVC',
                                               LinearSVC(random_state=0)),
                                              ('RF',
                                               RandomForestClassifier(n_estimators=250,
                                                                      random_state=0)),
                                              ('MLP',
                                               MLPClassifier(hidden_layer_sizes=(200,),
                                                             learning_rate_init=0.01,
                                                             random_state=0,
                                                             solver='sgd'))]))])

## Regresión

In [15]:
preprocessing = ColumnTransformer(
    transformers=[
        ('BoC-plat',boc_some_values,'platforms'),
        ('BoC-cat',boc_some_values,'categories'),
        ('BoC-genres',boc_some_values,'genres'),
        ('BoC-tags',boc_some_values,'tags'),

        ('BoC-dev',boc_many_values,'developer'),
        ('BoC-pub',boc_many_values,'publisher'),

        ('OneHotEncoder',OneHotEncoder(handle_unknown='ignore'),['month']),
        ('MinMaxScaler',MinMaxScaler(),['required_age','price','release_date']),
        ('BoxCox',PowerTransformer(method='yeo-johnson'),['achievements','average_playtime','revenue']),
        ('unchanged',Nothing(),['english'])
])

In [16]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import VotingRegressor

In [17]:
reg = VotingRegressor(estimators=[
        ('GB', GradientBoostingRegressor(n_estimators=25,random_state=0,learning_rate=0.1)),
        ('RF', RandomForestRegressor(n_estimators=250, random_state=0)),
        ('KN', KNeighborsRegressor(metric='euclidean', weights='distance')),
        ('MLP', MLPRegressor(hidden_layer_sizes=(200, 100),learning_rate='invscaling',random_state=0,max_iter=500))
    ])

In [18]:
%%capture
pipe_reg = make_pipeline(reg,preprocessing)
score = cross_val_score(pipe_reg,df_train, df_train['estimated_sells'],scoring='r2')

In [19]:
score

array([0.14044979, 0.23949368, 0.29040998, 0.42428286, 0.24468544])

In [20]:
np.mean(score)

0.26786435031204114

In [21]:
pipe_reg = make_pipeline(reg,preprocessing)
pipe_reg.fit(df_train, df_train['estimated_sells'])

/home/camilo/miniconda3/envs/prog_cientifica/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Pipeline(steps=[('procesamiento',
                 ColumnTransformer(transformers=[('BoC-plat',
                                                  CountVectorizer(min_df=0.05,
                                                                  tokenizer=<__main__.CategoriesTokenizer object at 0x7f40ac399c90>),
                                                  'platforms'),
                                                 ('BoC-cat',
                                                  CountVectorizer(min_df=0.05,
                                                                  tokenizer=<__main__.CategoriesTokenizer object at 0x7f40ac399c90>),
                                                  'categories'),
                                                 ('BoC-genres',
                                                  CountVectorizer(min_df=0.05,
                                                                  tokenizer...
                ('selector', SelectPercentile(percentile=95)),
                ('classifier',
                 VotingRegressor(estimators=[('GB',
                                              GradientBoostingRegressor(n_estimators=25,
                                                                        random_state=0)),
                                             ('RF',
                                              RandomForestRegressor(n_estimators=250,
                                                                    random_state=0)),
                                             ('KN',
                                              KNeighborsRegressor(metric='euclidean',
                                                                  weights='distance')),
                                             ('MLP',
                                              MLPRegressor(hidden_layer_sizes=(200,
                                                                               100),
                                                           learning_rate='invscaling',
                                                           max_iter=500,
                                                           random_state=0))]))])

### Generando archivo de submission

In [22]:
from zipfile import ZipFile
import os

def generateFiles(predict_data, clf_pipe, rgr_pipe):
    """Genera los archivos a subir en CodaLab

    Input
    predict_data: Dataframe con los datos de entrada a predecir
    clf_pipe: pipeline del clf
    rgr_pipe: pipeline del rgr

    Ouput
    archivo de txt
    """
    y_pred_clf = clf_pipe.predict(predict_data)
    y_pred_rgr = rgr_pipe.predict(predict_data)
    
    with open('./predictions_clf.txt', 'w') as f:
        for item in y_pred_clf:
            f.write("%s\n" % item)

    with open('./predictions_rgr.txt', 'w') as f:
        for item in y_pred_rgr:
            f.write("%s\n" % item)

    with ZipFile('predictions.zip', 'w') as zipObj2:
       zipObj2.write('predictions_rgr.txt')
       zipObj2.write('predictions_clf.txt')

    os.remove("predictions_rgr.txt")
    os.remove("predictions_clf.txt")

In [23]:
df_test = pd.read_pickle('test.pickle')
df_test = integrar_bert_logits(df_test)
df_test = custom_features(df_test)

In [24]:
generateFiles(df_test,pipe_clf,pipe_reg)